In [ ]:
import json
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def counts_to_percent_seen(pts_seen, n_total=200):
    n_fruit = len(pts_seen[0])
    pct_seen = np.stack(pts_seen) / n_total
    return pd.DataFrame({
        'any': np.sum(pct_seen > 0, axis=1)/n_fruit,
        '25%': np.sum(pct_seen >= 0.25, axis=1)/n_fruit,
        '50%': np.sum(pct_seen >= 0.25, axis=1)/n_fruit,
        '75%': np.sum(pct_seen >= 0.75, axis=1)/n_fruit,
    })

In [ ]:
with open('data/point_scanning.json') as f:
    data = json.load(f)

In [ ]:
for scenario in data:
    params = scenario['parameters']
    
    fig, axes = plt.subplots(2,3, figsize=(12, 8))  # Create a figure with 3 subplots    
    plt.suptitle('Leaf density: {}; seed: {}'.format(params['tree_params']['leaf_scale'], params['tree_params']['seed']))
    
    columns = ['any','50%','75%']
    
    for attempt in scenario['attempts']:
        
        att_params = attempt['orbit']
        trace = pd.json_normalize(attempt['result']['frames'])
        trace['distance_from_last'] = np.stack(trace['joint_distances.joint_distances']).sum(axis=1) + trace['joint_distances.rotation_distance'] + trace['joint_distances.translation_distance']
        trace['distance'] = trace['distance_from_last'].cumsum()
        
        pct_seen = counts_to_percent_seen(trace['pts_seen'])
        
        for (ax,col) in zip(axes[0],columns):
            ax.plot(trace['distance'], pct_seen[col], label=att_params['parameters']['parameters']['radius'])

            
        pct_seen = counts_to_percent_seen(trace['interior_pts_seen'])
        
        for (ax,col) in zip(axes[1],columns):
            ax.plot(trace['distance'], pct_seen[col], label=att_params['parameters']['parameters']['radius'])
        
    for ax, col in zip(axes[0], columns):
        ax.set_ylim(0,1.1)
        ax.grid()
        ax.set_xlabel('Path length')
        ax.set_ylabel('Proportion of apples')
        ax.set_title('{} points seen'.format(col))
        ax.legend()

    for ax, col in zip(axes[1], columns):
        ax.set_ylim(0,1.1)
        ax.grid()
        ax.set_xlabel('Path length')
        ax.set_ylabel('Proportion of apples')
        ax.set_title('{} interior points seen'.format(col))
        ax.legend()

    fig.tight_layout()

In [ ]:
trace['joint_distances.rotation_distance'].cumsum().plot()